In [22]:
import pandas as pd, dataprep as dp
import torch
import torch.nn as nn
import numpy as np

In [2]:
filename = "C:/Users/paulh/Desktop/6.semester/Iot/IoT/main/agg_hourly.parquet"
data = dp.prepare_data(filename, window_size= 50, train_ratio= 0.8, batch_size= 64, features= ["tmp", "CO2", "hum", "VOC"], target= "tmp", scaling= True)
data

{'train': Data(x=tensor([[[ 0.1110,  0.0978,  1.1618, -1.2062],
          [ 0.3710, -0.0261,  0.7302, -1.1759],
          [ 0.3892, -0.1984,  0.8281, -1.0404],
          ...,
          [ 0.7391, -0.5215,  0.2090, -0.4050],
          [ 1.2377, -0.5538, -0.0441, -0.3350],
          [ 0.6267, -0.6184,  0.4645, -0.4915]],
 
         [[ 0.3710, -0.0261,  0.7302, -1.1759],
          [ 0.3892, -0.1984,  0.8281, -1.0404],
          [ 0.3903, -0.1123,  1.0159, -0.9680],
          ...,
          [ 1.2377, -0.5538, -0.0441, -0.3350],
          [ 0.6267, -0.6184,  0.4645, -0.4915],
          [ 0.4071, -0.5969,  0.6948, -0.5288]],
 
         [[ 0.3892, -0.1984,  0.8281, -1.0404],
          [ 0.3903, -0.1123,  1.0159, -0.9680],
          [ 0.3798, -0.0099,  1.3002, -0.5522],
          ...,
          [ 0.6267, -0.6184,  0.4645, -0.4915],
          [ 0.4071, -0.5969,  0.6948, -0.5288],
          [ 0.3145, -0.5592,  0.8905, -0.4751]],
 
         ...,
 
         [[ 1.7935, -0.5053,  0.0133,  0.5831],
  

In [3]:
data["train"].x.shape[2]

4

In [67]:
from foo import PositionalEncoding, MultiHeadAttention, ScaledDotProduct, Decoder 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Decoder(input=data["train"].x.shape[2], d_model=64, max_len=50, num_heads= 4, d_ff= 100,device= device)

In [83]:
from tqdm import tqdm

num_epochs = 15
learning_rate = 0.0001
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
train_loader = data["train"].loader

model.train()
for epoch in range(num_epochs):
    loop = tqdm(enumerate(train_loader), total=len(train_loader), leave=True)
    for i, (features, target) in loop:
        features = features.to(device)
        target = target.to(device)
        optimizer.zero_grad(set_to_none=True)
        output = model(features)
        target = target.squeeze(-1)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        # Update progress bar
        loop.set_description(f"Epoch [{epoch+1}/{num_epochs}]")
        loop.set_postfix(loss=loss.item(), lr= learning_rate)
    

# Final output
print(f"Training completed. Final loss: {loss.item()}, final learning rate: {learning_rate}")

Epoch [5/5]: 100%|██████████| 443/443 [00:09<00:00, 46.45it/s, loss=0.0242, lr=0.0001]  

Training completed. Final loss: 0.024172496050596237, final learning rate: 0.0001


In [84]:
model.eval()  
test_loader = data["test"].loader

test_features, test_targets = next(iter(test_loader)) 
test_targets = test_targets.squeeze(-1)  
print(test_features.shape)
print(test_targets.shape)
test_features = test_features.to(device)
test_targets = test_targets.to(device)

with torch.no_grad():  
    predictions = model(test_features)  

train_loss = criterion(predictions, test_targets)
print('Test Loss: {:.4f}'.format(train_loss.item()))

torch.Size([64, 50, 4])
torch.Size([64])
Test Loss: 0.0178


In [85]:
from sklearn.preprocessing import StandardScaler
feature_index = 0
scaler = data["test"].scaler


feature_scaler = StandardScaler()
feature_scaler.mean_ = scaler.mean_[feature_index]
feature_scaler.scale_ = scaler.scale_[feature_index]

inversed_predictions = feature_scaler.inverse_transform(predictions.to("cpu").numpy().reshape(-1, 1))
inversed_targets = feature_scaler.inverse_transform(test_targets.to("cpu").numpy().reshape(-1, 1))

In [86]:
import plotly.graph_objects as go

def plt_pred(test_targets, predictions):
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=test_targets, mode='lines', name='test_targets'))
    fig.add_trace(go.Scatter(y=predictions, mode='lines', name='Predictions'))
    fig.update_layout(title='test_targets vs predictions', xaxis_title='Index', yaxis_title='Value')
    fig.show()

plt_pred(inversed_targets.reshape(-1).tolist(), inversed_predictions.reshape(-1).tolist())

In [82]:
torch.save(model.state_dict(), "transformer_gut.pth")